# Prétraitement avancé pour la classification de texte

Ce notebook détaille un pipeline complet de prétraitement pour la classification de texte. Il explique chaque étape, du chargement des données à la sauvegarde des matrices prêtes pour l'apprentissage automatique.

Nous utilisons :
- Nettoyage et lemmatisation avec SpaCy
- Vectorisation avec CountVectorizer
- Encodage des labels
- Padding/truncation des matrices
- Sauvegarde des artefacts pour un usage ultérieur

---

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import json
import re
import spacy
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Version et introduction

On affiche la version du script et on explique le but : préparer les données pour un modèle de classification de texte.

In [ ]:
print("V2_1.1.0 – Prétraitement train + test et sauvegarde")

## 1. Chargement des données d'entraînement

On charge les données d'entraînement depuis un fichier JSON. Chaque ligne correspond à un texte à classer. On remplace les valeurs manquantes par une chaîne vide pour éviter les erreurs lors du traitement.

In [ ]:
print("Loading training data...")
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df = pd.DataFrame(train_data).fillna('')

## Fusion des labels d'entraînement

On associe à chaque texte sa catégorie cible (label) pour l'apprentissage supervisé.

In [ ]:
print("Merging train labels...")
labels_df = pd.read_csv('train_label.csv')
df = df.merge(labels_df, on='Id')

## 2. Chargement des données de test

On charge les textes à prédire (test) et on remplace aussi les valeurs manquantes.

In [ ]:
print("Loading test data...")
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
test_df = pd.DataFrame(test_data).fillna('')

## 3. Nettoyage et lemmatisation des textes

On utilise SpaCy pour nettoyer et lemmatiser les textes :
- Suppression du HTML et des URLs
- Mise en minuscules
- Lemmatisation (réduction à la racine)
- Suppression des stopwords et tokens non alphabétiques

Ce prétraitement réduit la dimensionnalité et améliore la qualité des features pour le modèle.

In [ ]:
print("Cleaning and lemmatizing text data...")
nlp = spacy.load('en_core_web_sm')
def clean_text(text):
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = text.lower().strip()
    doc = nlp(text)
    return ' '.join(tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop)

df['Clean']      = df['description'].apply(clean_text)
test_df['Clean'] = test_df['description'].apply(clean_text)

## 4. Encodage des labels d'entraînement

On transforme les catégories textuelles en entiers avec `LabelEncoder`. Cela permet de les utiliser comme cibles pour l'apprentissage automatique.

On prépare aussi le nombre de classes pour le one-hot encoding.

In [ ]:
print("Encoding train labels...")
le = LabelEncoder()
df['y'] = le.fit_transform(df['Category'])
num_classes = len(le.classes_)

## 5. Vectorisation des textes (CountVectorizer)

On transforme les textes en vecteurs de nombres avec `CountVectorizer` :
- Chaque texte devient un vecteur de taille `max_features` (ici 20 000)
- Chaque dimension correspond à un mot du vocabulaire
- La valeur est le nombre d'occurrences du mot dans le texte

Mathématiquement, cela revient à construire une matrice creuse (sparse) de taille (n_samples, vocab_size), où chaque ligne représente un texte et chaque colonne un mot du vocabulaire.

In [ ]:
print("Vectorizing train + test (CountVectorizer)...")
MAX_VOCAB = 20000
MAX_LEN   = 100

vectorizer = CountVectorizer(max_features=MAX_VOCAB, stop_words='english')
X_train = vectorizer.fit_transform(df['Clean']).toarray()
X_test  = vectorizer.transform(test_df['Clean']).toarray()

## 6. Padding/Truncation des matrices

Pour garantir que tous les vecteurs aient la même taille (`MAX_LEN`), on tronque ou complète les vecteurs :
- Si le vecteur est trop long, on coupe
- S'il est trop court, on complète avec des zéros

Cela permet d'utiliser ces matrices comme entrée pour des modèles de deep learning ou d'autres algorithmes qui exigent une taille fixe.

In [ ]:
def pad_trunc(mat, max_len):
    if mat.shape[1] > max_len:
        return mat[:, :max_len]
    elif mat.shape[1] < max_len:
        return np.pad(mat, ((0,0),(0, max_len - mat.shape[1])), mode='constant')
    else:
        return mat

X_train = pad_trunc(X_train, MAX_LEN)
X_test  = pad_trunc(X_test,  MAX_LEN)

## 7. One-hot encoding des labels d'entraînement

On transforme les labels en vecteurs one-hot :
- Pour chaque exemple, le vecteur est de taille `num_classes`
- La case correspondant à la classe est à 1, les autres à 0

Cela facilite l'entraînement de modèles multi-classes.

In [ ]:
y_train = np.eye(num_classes)[df['y']]

## 8. Sauvegarde des artefacts

On sauvegarde :
- Le vectorizer (pour transformer les nouveaux textes de la même façon)
- Le label encoder (pour décoder les prédictions)
- Les matrices numpy prêtes pour l'entraînement ou l'inférence

Cela permet de réutiliser exactement le même prétraitement lors de la prédiction.

In [ ]:
print("Saving vectorizer, label encoder, X_train.npy, y_train.npy, X_test.npy …")
joblib.dump(vectorizer,    'vectorizer.joblib')
joblib.dump(le,            'label_encoder.joblib')
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_test.npy',  X_test)

## Fin du pipeline

Le prétraitement est terminé. Les fichiers sauvegardés peuvent être utilisés pour entraîner un modèle de classification de texte (réseau de neurones, SVM, etc.).

Ce pipeline garantit que le texte brut est transformé en vecteurs numériques exploitables par les algorithmes d'apprentissage automatique, tout en conservant la correspondance entre les textes et leurs catégories.

In [ ]:
print("✅ Prétraitement terminé et fichiers sauvegardés.")